<a href="https://colab.research.google.com/github/Vaibhavs10/scratchpad/blob/main/langchain_faiss_hf_api_repro.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install huggingface_hub chromadb langchain unstructured unstructured[local-inference] faiss-cpu

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.0/17.0 MB 49.1 MB/s eta 0:00:00


In [2]:
from langchain.document_loaders import OnlinePDFLoader

loader = OnlinePDFLoader("https://arxiv.org/pdf/2304.03757.pdf")

In [3]:
documents = loader.load()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


In [4]:
from langchain.text_splitter import CharacterTextSplitter
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_documents(documents)

In [6]:
from langchain.llms import HuggingFaceHub

flan_ul2 = HuggingFaceHub(repo_id="google/flan-ul2", model_kwargs={"temperature":0.1, "max_new_tokens":300})

In [7]:
from langchain.embeddings import HuggingFaceHubEmbeddings

embeddings = HuggingFaceHubEmbeddings()

You're using a different task than the one specified in the repository. Be sure to know what you're doing :)


In [8]:
from langchain.vectorstores import FAISS
db = FAISS.from_documents(texts, embeddings)

In [9]:
retriever = db.as_retriever()

In [10]:
from langchain.chains import RetrievalQA

qa = RetrievalQA.from_chain_type(llm=flan_ul2, chain_type="stuff", retriever=retriever)

In [11]:
query = "What is the title of this paper?"
qa.run(query)

ValueError: ignored